[View in Colaboratory](https://colab.research.google.com/github/hyerim1048/TodayILearned/blob/master/pytorch_RL.ipynb)

In [2]:
!pip3 install torch gym

    100% |████████████████████████████████| 484.0MB 24kB/s 
tcmalloc: large alloc 1073750016 bytes == 0x5b354000 @  0x7fdea02271c4 0x46d6a4 0x5fcbcc 0x4c494d 0x54f3c4 0x553aaf 0x54e4c8 0x54f4f6 0x553aaf 0x54efc1 0x54f24d 0x553aaf 0x54efc1 0x54f24d 0x553aaf 0x54efc1 0x54f24d 0x551ee0 0x54e4c8 0x54f4f6 0x553aaf 0x54efc1 0x54f24d 0x551ee0 0x54efc1 0x54f24d 0x551ee0 0x54e4c8 0x54f4f6 0x553aaf 0x54e4c8
    100% |████████████████████████████████| 1.5MB 23.3MB/s 
    100% |████████████████████████████████| 1.0MB 21.0MB/s 
  Running setup.py bdist_wheel for gym ... - \ | / done
  Stored in directory: /content/.cache/pip/wheels/cb/14/71/f4ab006b1e6ff75c2b54985c2f98d0644fffe9c1dddc670925
Successfully built gym


In [2]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F
import numpy as np
import gym

In [3]:
BATCH_SIZE = 32 # batch 크기
LR = 0.01 # learning rate
EPSILON = 0.9  # greedy policy
GAMMA = 0.9  # discount factor
TARGET_REPLACE_ITER = 100   # target update frequency
MEMORY_CAPACITY = 2000


## openAI 의 gym 패키지

강화학습 환경을 쉽게 구현해주는 패키지.

### Wrappers
enviroment 를 modular way로 변형
self.unwrapped : access the inner enviroment

In [4]:
env = gym.make('CartPole-v0') # <TimeLimit<CartPoleEnv<CartPole-v0>>>
env = env.unwrapped # gym.envs.classic_control.cartpole.CartPoleEnv
N_ACTIONS = env.action_space.n # action space 2torch.unsqueeze(torch.FloatTensor(x), 0)
N_STATES = env.observation_space.shape[0] # 4
# http://daeson.tistory.com/311
# http://jaynewho.com/post/10
# https://github.com/openai/gym/tree/b9ef43cb387a93af9728b0fc06cb1723f71bc497/gym/wrappers
# 

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


## torch.squeeze(input, dim=None, out=None) -> Tensor
* 차원에서 1size가 제거됨.
* ex) (A,1,B,C) -> (A,B,C)
* dim이 주어지면, 주어진 dimension에서만 squeeze 함.

## torch.unsqueeze(input, dim,out=None) -> tensor
* Returns a new tensor with a dimension of size one inserted at the specified position.

In [17]:
torch.unsqueeze(torch.FloatTensor([1,1]), 0)

tensor([[ 1.,  1.]])

In [5]:
class Net(nn.Module):
  def __init__(self,):
    super(Net, self).__init__()
    self.fc1 = nn.Linear(N_STATES,10)  
    self.fc1.weight.data.normal_(0,)
    self.out = nn.Linear(10,N_ACTIONS) # action 2  가지 방향
    self.out.weight.data.normal_(0,0.1)
    
  def forward(self,x):
    x.self.fc1(x)
    x = self.fc1(x)
    x = F.relu(x)
    actions_value = self.out(x)
    return actions_value

In [0]:
class DQN(object):
    def __init__(self):
        self.eval_net, self.target_net = Net(), Net()

        self.learn_step_counter = 0                                     # for target updating
        self.memory_counter = 0                                         # for storing memory
        self.memory = np.zeros((MEMORY_CAPACITY, N_STATES * 2 + 2))     # initialize memory # 2000*10 why 10?
        self.optimizer = torch.optim.Adam(self.eval_net.parameters(), lr=LR) # optimizer는 아담을 사용함
        self.loss_func = nn.MSELoss() # loss function은 그냥 MSE

    def choose_action(self, x):
        x = Variable(torch.unsqueeze(torch.FloatTensor(x), 0)) # x를 받아서 맨처음에 size 1차원을 왜 늘리지?
        # input only one sample
        if np.random.uniform() < EPSILON:   # greedy # random 0 ~ 1뽑아서 그게 epsilon 보다 작으면,
            actions_value = self.eval_net.forward(x) # x를 뉴럴 넷에 넣고 action value값을 받음.
            action = torch.max(actions_value, 1)[1].data.numpy()[0, 0]     # return the argmax
        else:   # random
            action = np.random.randint(0, N_ACTIONS) # 1 - epsilon 확률로 랜덤하게 움직인다.
        return action



In [0]:
dqn = DQN()

In [0]:
print('\nCollecting experience...')
for i_episode in range(400): # episode 는 400번
    s = env.reset() # env 초기화
    ep_r = 0
    while True:
        env.render()
        a = dqn.choose_action(s) # action 값을 선택함 deep neural network + epsilon greedy 방식 에 따라서

        # take action
        s_, r, done, info = env.step(a) # a를 넣으면 
